In [ ]:
import numpy as np
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
import datetime

import time

try:
    from googlesearch import search
except ImportError:
    print("No module named google found")

In [ ]:
df=pd.read_csv("Nifty50Links.csv",index_col=0)

In [ ]:
names=df['Company Name'].values
symbols=df['NSESymbol'].values

In [ ]:
StandAlonelinks=[]

for i in range(len(names)):
    query=names[i]+" moneycontrol quarterly results"
    for j in search(query,tld='co.in',num=1,pause=2):
        StandAlonelinks.append(j)
        print(j)
        break
        #As the first result shown in google is what we require

In [ ]:
def NetProfitScraper(i):
    global soup
    
    page=requests.get(StandAlonelinks[i])
    print('page created')
    soup=bs(page.content,'lxml')
    print('soup object created')
    for link in soup.findAll('a',attrs={'href':re.compile("^.*(consolidated).*$",re.IGNORECASE)}):
        print('Returning link for',i)
        return link

Consolidatedlinks=[]

for i in range(len(StandAlonelinks)):
    Consolidatedlinks.append(NetProfitScraper(i))
   

In [ ]:
import os
path=r'C:\Users\User\Desktop\Earnings Data files\\'
os.mkdir(path)
for i in names:
    path=os.path.join(r'C:\Users\User\Desktop\Earnings Data files\\'+i)
    os.mkdir(path)
    print('Directory %s is created'%i)

In [ ]:
def scrapeData(i,count):
    global quarters,pnl
    
    temp=i.split('?')
    i=temp[0]+'/'+str(count)
    #print(i)
    
    page=requests.get(i)
    soup=bs(page.content,'lxml')
    
    
    #print('soup object created')
    
    
    
    table=soup.find('table',attrs={'class':'mctable1'})
    
    table_rows=table.findAll('tr')
    
    
    #print('Extracting data')
    for tr in table_rows:
        td=tr.findAll('td')
        
        try:
            row=[tr.text for tr in td]
                    
            
            if(re.compile(r'^Quarterly').match(row[0])):
                for t in range(len(row)-1):
                    quarters.append(row[t])
                
            elif(re.compile(r'^Net Profit').match(row[0])):
                for t in range(len(row)-1):
                    pnl.append(row[t])
                
        except(IndexError):
            continue
    
    
        

In [ ]:
path=r'C:\Users\User\Desktop\Earnings Data files\\'
now=datetime.datetime.now()
preY=now.year
check=((preY-2012)*4)/5#Condition needed as we scrapped dates of quarterly results only upto 2012 

error_list=[]
#len(StandAlonelinks
for i in range(len(StandAlonelinks)):
    try:
        link=StandAlonelinks[i]
        df=pd.DataFrame(columns=['Quarters','PnL(in crores)'])
        quarters=[]
        
        pnl=[]


        counter=0
        while(counter<check):
            counter+=1
            scrapeData(link,counter)
            time.sleep(1)

        print(StandAlonelinks[i],' is done\n',len(StandAlonelinks)-i-1,' :left')
        df['Quarters']=quarters
        df['PnL']=pnl

        df=df.drop_duplicates(subset='Quarters',keep='first')
        df=df.drop(0,axis=0)

        df.to_csv(r''+path+names[i]+'\\data.csv')
    except(AttributeError):
        error_list.append(link)
        print(StandAlonelinks[i],' has error\n',len(StandAlonelinks)-i-1,' :left')
        continue
        

In [ ]:
#Contains list of all stocks with problems
error_list

# Storing data for consolidated stocks

In [ ]:
import os
path=r'C:\Users\User\Desktop\Earnings Data(consolidated) files\\'
os.mkdir(path)
for i in names:
    path=os.path.join(r'C:\Users\User\Desktop\Earnings Data(consolidated) files\\'+i)
    os.mkdir(path)
    print('Directory %s is created'%i)

In [ ]:
path=r'C:\Users\User\Desktop\Earnings Data(consolidated) files\\'
now=datetime.datetime.now()
preY=now.year
check=((preY-2012)*4)/5#Condition needed as we scrapped dates of quarterly results only upto 2012 

error_list2=[]
for i in range(len(Consolidatedlinks)):
    try:
        link=Consolidatedlinks[i]
        df=pd.DataFrame(columns=['Quarters','PnL(in crores)'])
        quarters=[]
        pnl=[]


        counter=0
        while(counter<check):
            counter+=1
            scrapeData(link,counter)
            time.sleep(1)

        print(Consolidatedlinks[i],' is done\n',len(Consolidatedlinks)-i-1,' :left')
        df['Quarters']=quarters
        df['PnL']=pnl

        df=df.drop_duplicates(subset='Quarters',keep='first')
        df=df.drop(0,axis=0)

        df.to_csv(r''+path+names[i]+'\\data.csv')
    except(AttributeError):
        error_list.append(link)
        print(Consolidatedlinks[i],' has error\n',len(Consolidatedlinks)-i-1,' :left')
        continue

In [ ]:
#Contains list of all stocks with problems
error_list2

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import re
import datetime
from datetime import date,datetime
import os

df=pd.read_csv('ind_nifty50list.csv',index_col=[0])#Please make sure you have an updated list of nifty50 stocks from NSE website



nstocks=0#Will contain final number of stocks considered

attr1={'class':'table-txt-right'}
attr2={'class':'table table-BG-grey mT20'}#This is used to find the table containing Board meeting announcements

finaldf=pd.read_csv('Nifty50Links.csv',index_col=[0])


In [ ]:
def quarterCheck(tempD):
    tempD=datetime.strptime(tempD,'%d-%m-%Y')
    tempD=tempD.date()
    
    m=tempD.month
    y=tempD.year
    
    if(m==1 or m==2 or m==3):
        return('Dec \''+str(y-1))
    elif(m==4 or m==5 or m==6):
        return('Mar \''+str(y))
    elif(m==7 or m==8 or m==9):
        return('Jun \''+str(y))
    elif(m==10 or m==11 or m==12):
        return('Sep \''+str(y))
    

#This Function accepts link and data. Then it scrapes all the announcements made and searches for keywords if keyword
# is found then it is classified accordingly. Then its date is recorded to see which month it belongs to and then calculates 
# the previous quarter using quarterCheck function

def newsDeclare(templink2,name,path):
    global errorlist
    
    
    page=requests.get(templink2)
    soup=bs(page.content,'lxml')
    
    
    
    table2=soup.find('div',attrs=attr2)
    
    table2rows=table2.find_all('tr')
    
    text=[]
    
    
    for tr in table2rows:

        td=tr.find_all('td')
#        
        temp=''
        row=[tr.text for tr in td]

        temp=tr.find('span',attrs={'title':re.compile(r'^.*(unaudited|audited)*.$',re.IGNORECASE)})

        if(str(temp)!='None' and row[0]!='ANNOUNCEMENT DATE'):

            text.append([row[0],quarterCheck(row[0]),'Audited Results Announcement'])

        elif(row[0]!='ANNOUNCEMENT DATE'):

            temp=tr.find('span',attrs={'title':re.compile(r'^.*quarterly*.$',re.IGNORECASE)})
            if(str(temp)!='None'):
                text.append([row[0],quarterCheck(row[0]),'Quarterly Results Announcement'])




        
    tempdf=pd.DataFrame(text,columns=['Date of Announcement','Quarters','Announcement Type'])
    tempdf=tempdf.drop_duplicates(subset='Quarters',keep='first')
        #print(tempdf)



    try:
        Edf=pd.read_csv(path+name+'\data.csv',index_col=[0])

        tempdf=tempdf.drop_duplicates('Quarters',keep='last')
        Edf=Edf.drop('PnL(in crores)',axis=1)



        datesLen=len(tempdf)
        Edf=Edf.drop(Edf.index[datesLen:],axis=0)
        Edf['Date of Announcement']=tempdf['Date of Announcement'].values
        Edf['Announcement Type']=tempdf['Announcement Type'].values


        os.remove(path+name+'\data.csv')
        Edf.to_csv(path+name+'\data.csv')
        
    except:
        print(name,':Error')
        pass



In [ ]:
finalLink=[]
link="https://www.business-standard.com"#This is the website to which extension is added
link3="/corporate-action?purpose=board&fromDay=&fromMonth=&fromYear=&toDay=&toMonth=&toYear="#Used to get boardmeeting page


In [ ]:
#We now finally have the list of websites for Nifty-50 stocks containing information regarding Board Meetings
for i in range(len(finaldf)):
    temp=(link+finaldf['Links'][i])
    temp=temp.split('.html')
    finalLink.append(temp[0]+link3)
    

In [ ]:
#This is the final loop which helps create csvs containing data of announcements with their company names.


for i in range(5,len(finalLink)):
    path1=r'C:\Users\User\Desktop\Earnings Data files\\'#for standalone
    path2=r'C:\Users\User\Desktop\Earnings Data(Consolidated) files\\'#for consolidated
    newsDeclare(finalLink[i],finaldf['Company Name'][i],path1)
    print('Companies left:',len(finalLink)-i-1)
    